In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("Gruobi")
Pkg.add("DataFrames")
Pkg.add("CSV")

In [1]:
using JuMP, GLPK
using DataFrames
using CSV

In [2]:
#load demand data
demand = CSV.File("../data/demand.csv") |> DataFrame

#load cost data
cost = CSV.File("../data/cost_generation.csv") |> DataFrame

#load capacity data
capacity = CSV.File("../data/technical_data_generation.csv") |> DataFrame

#load wind_technicaldata
wind = CSV.File("../data/wind_technicaldata.csv") |> DataFrame


Row,Unit,Node,Pmax (MW),Pmin (MW),Ci ($/MWh)
,Int64,Int64,Int64,Int64,Int64
1,1,3,200,0,0
2,2,5,200,0,0
3,3,7,200,0,0
4,4,16,200,0,0
5,5,21,200,0,0
6,6,23,200,0,0


In [3]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:(size(cost, 1)+size(wind, 1))
    result_df[!, Symbol("x$i")] = Float64[]
end

In [3]:

for hour in 1:24
  # Create a new model with GLPK solver
  model = Model(GLPK.Optimizer)

  # Define the decision variables for every generator
  unregister(model, :x)  # Unregister the existing variable named "x" from the model
  
  #demand bids
  name = "demand_hour_" * string(hour-1) * ".csv"
  demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
   
  # number of convential generators
   G = 1:size(cost, 1)

   # number of demand
   D = 20
 
   # number of wind generators
   W = 1:size(wind, 1)


  # decision variables for convential generators
  @variable(model, x[1:size(cost, 1)])  # Create the new variable
  for generator in 1:size(cost, 1)
      set_lower_bound(x[generator], capacity[generator,4])  # Set the lower bound
      set_upper_bound(x[generator], capacity[generator,3])  # Set the upper bound
  end

  # wind decision variables
  @variable(model, x_wind[1:size(wind, 1)])  # Create the new variable
  for generator in 1:size(wind, 1)
      set_lower_bound(x_wind[generator], wind[generator,4])  # Set the lower bound
      set_upper_bound(x_wind[generator], wind[generator,3])  # Set the upper bound
  end

  # decision variables for demand
  @variable(model, y[1:size(demand_bids, 1)])  # Create the new variable
  for bid in 1:size(demand_bids, 1)
      set_lower_bound(y[bid], 0)  # Set the lower bound
      set_upper_bound(y[bid], demand_bids[bid,2])  # Set the upper bound
  end


  # Define the objective function
  @objective(model, Max, sum(Uᴰ[d] * pᴰ[d] for d in D) - sum(Cᴳ[g] * pᴳ[g] for g in G))

  # Add the constraint for the balance between supply and demand
  @constraint(model, balance, sum(pᴰ[d] for d in D) - sum(pᴳ[g] for g in G) == 0)

  # Print the model
  print(model)

  # Solve the model
  optimize!(model)

  # Check the status of the solution
  status = termination_status(model)
  if status == MOI.OPTIMAL
      println("Hour ", hour, ": Optimal solution found")

      # Assuming 'model' and 'x' are properly defined and 'hour' is known
      new_row = [hour, objective_value(model)]
      append!(new_row, [value(x[i]) for i in 1:size(cost, 1)]) # Simplifies adding x values

      push!(result_df, new_row)

  elseif status == MOI.INFEASIBLE
      println("Hour ", hour, ": No feasible solution found")
  else
      println("Hour ", hour, ": Solver status: ", status)
  end
end

ErrorException: syntax: character literal contains multiple characters

In [4]:

    @variable(model, x[1:size(cost, 1)])  # Create the new variable
    for generator in 1:size(cost, 1)
        set_lower_bound(x[generator], capacity[generator,4])  # Set the lower bound
        set_upper_bound(x[generator], capacity[generator,3])  # Set the upper bound
    end


    # Define the objective function
    @objective(model, Min, sum(cost[generator,2] * x[generator] for generator in 1:size(cost,1)))

    # Add the equilibrium constraint
    @constraint(model, sum(x[generator] for generator in 1:size(cost,1)) == demand[hour,2])

Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to


 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 1775.835
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350


Hour 1: Optimal solution found


Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 1669.815
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 

x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 2: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 1590.3
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60


...

 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 19: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 2544.48
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 20: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 2411.955
 x[1] >= 30


 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 21: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 2199.915
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591


 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 22: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 1934.865
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155


 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 23: Optimal solution found
Min 13.32 x[1] + 13.32 x[2] + 20.7 x[3] + 20.93 x[4] + 26.11 x[5] + 10.52 x[6] + 10.52 x[7] + 6.02 x[8] + 5.47 x[9] + 10.52 x[11] + 10.89 x[12]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] == 1669.815
 x[1] >= 30.4
 x[2] >= 30.4
 x[3] >= 75
 x[4] >= 206.85
 x[5] >= 12
 x[6] >= 54.25
 x[7] >= 54.25
 x[8] >= 100
 x[9] >= 100
 x[10] >= 300
 x[11] >= 108.5
 x[12] >= 140
 x[1] <= 152
 x[2] <= 152
 x[3] <= 350
 x[4] <= 591
 x[5] <= 60
 x[6] <= 155
 x[7] <= 155
 x[8] <= 400
 x[9] <= 400
 x[10] <= 300
 x[11] <= 310
 x[12] <= 350
Hour 24: Optimal solution found


In [5]:
result_df

Row,hour,objective_value,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,15192.9,30.4,30.4,75.0,206.85,12.0,54.25,54.25,364.185,400.0,300.0,108.5,140.0
2,2,14554.6,30.4,30.4,75.0,206.85,12.0,54.25,54.25,258.165,400.0,300.0,108.5,140.0
3,3,14076.0,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
4,4,13916.4,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
5,5,13916.4,30.4,30.4,75.0,206.85,12.0,54.25,54.25,152.145,400.0,300.0,108.5,140.0
6,6,14076.0,30.4,30.4,75.0,206.85,12.0,54.25,54.25,178.65,400.0,300.0,108.5,140.0
7,7,16983.5,30.4,30.4,75.0,206.85,12.0,155.0,103.22,400.0,400.0,300.0,108.5,140.0
8,8,20353.5,30.4,30.4,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,204.78
9,9,23178.0,30.4,123.725,75.0,206.85,12.0,155.0,155.0,400.0,400.0,300.0,310.0,350.0


In [ ]:
#save result_df to csv
CSV.write("../results/market_clearing_Jan.csv", result_df)